This file is based on "4.27- Link input with activations.ipynb", with updated data structure (different list structures -> list of tuples). There are also more data that are useful for making visualizations

1. Load libraries

In [143]:
from PIL import Image
import numpy as np
import os
import pickle
import random
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from torchvision import transforms

# Use IoU instead
# import re
from scipy.spatial.distance import cdist
from skimage.transform import resize

2. Decide the procedure by grouping the different columns together

For cityscapes and synthia separately (and then concatenate):

* name, image_path, dataset

* label_path:
    * first get labels (in class format)
        * for synthia, need to transform to class format specifically
            * for calculating similarity
        * also from class to color as well, because use the color of cityscapes
            * for saving the label and getting the label path
        * get the saved path and save it in a list
    
* similar_image_paths，similar_IoU_score (Note: called IoU_score originally): finding the most similar masks from another dataset
* Generate from os list of images:
    * class distribution
        * (other_ratio, road_ratio, sidewalk_ratio, vegetation_ratio, sky_ratio, car_ratio)
    * embedding of input (originally: tsne_1, tsne_2)
        simple dimensionality reduction (simple_tsne_1,simple_tsne_2)
        and also the embedding from classification model(meaningful_tsne_1,meaningful_tsne_1)
* bottleneck_activations_embedding, prediction_path, and performance
    * save output locally, and then add path (prediction_path)
    * performance: (other_IoU, road_IoU, sidewalk_IoU, vegetation_IoU, sky_IoU, car_IoU)
        **How do you get the IoU per class (check original paper)**

**Make sure that the column names are the same as current！！！**

**Remember to also save id**

# name, image_path, dataset

In [2]:
# path of the folders
image_path_cityscapes = "D:\zsh\graduation\Graduation-project-domain-shift-image-2-image\dataset\original_cityscapes_inputs"
image_path_synthia = "D:\zsh\graduation\Graduation-project-domain-shift-image-2-image\dataset\SYNTHIA_256\image"

start = "D:\zsh\graduation\Graduation-project-domain-shift-image-2-image"
relative_img_path_cityscapes = os.path.relpath(image_path_cityscapes, start)
relative_img_path_synthia = os.path.relpath(image_path_synthia, start)


In [3]:
sample_number = 100

In [4]:
cityscapes_names= os.listdir(image_path_cityscapes)
random.seed(55)
cityscapes_names_sample = random.sample(cityscapes_names,sample_number)

synthia_names = os.listdir(image_path_synthia)
# randomly select 100 images to load in to numpy array
random.seed(55)
synthia_names_sample = random.sample(synthia_names,sample_number)

In [5]:
# Cityscapes
cityscapes_initial_info = []
cityscapes_img_sample = []

for name in cityscapes_names_sample:
    image = Image.open(image_path_cityscapes+"\\"+name).convert("RGB")
    image_path = relative_img_path_cityscapes+"\\"+name
    
    cityscapes_img_sample.append(np.array(image))
    cityscapes_initial_info.append((name,image_path,"Cityscapes"))

cityscapes_initial_info = np.array(cityscapes_initial_info)

In [8]:
cityscapes_initial_info[:5]

array([['14.jpeg', 'dataset\\original_cityscapes_inputs\\14.jpeg',
        'Cityscapes'],
       ['87.jpeg', 'dataset\\original_cityscapes_inputs\\87.jpeg',
        'Cityscapes'],
       ['189.jpeg', 'dataset\\original_cityscapes_inputs\\189.jpeg',
        'Cityscapes'],
       ['167.jpeg', 'dataset\\original_cityscapes_inputs\\167.jpeg',
        'Cityscapes'],
       ['439.jpeg', 'dataset\\original_cityscapes_inputs\\439.jpeg',
        'Cityscapes']], dtype='<U43')

In [13]:
# synthia

synthia_initial_info = []
synthia_img_sample = []

for name in synthia_names_sample:
    image = Image.open(image_path_synthia+"\\"+name).convert("RGB")
    image_path = relative_img_path_synthia+"\\"+name
    synthia_initial_info.append((name, image_path,"Synthia"))
    synthia_img_sample.append(np.array(image))

synthia_initial_info = np.array(synthia_initial_info)
synthia_img_sample = np.array(synthia_img_sample)

In [18]:
synthia_initial_info[:5]

array([['0001480.png', 'dataset\\SYNTHIA_256\\image\\0001480.png',
        'Synthia'],
       ['0003215.png', 'dataset\\SYNTHIA_256\\image\\0003215.png',
        'Synthia'],
       ['0002457.png', 'dataset\\SYNTHIA_256\\image\\0002457.png',
        'Synthia'],
       ['0004959.png', 'dataset\\SYNTHIA_256\\image\\0004959.png',
        'Synthia'],
       ['0001305.png', 'dataset\\SYNTHIA_256\\image\\0001305.png',
        'Synthia']], dtype='<U37')

In [19]:
combined_initial_info = np.concatenate((cityscapes_initial_info,synthia_initial_info),axis=0)

In [23]:
df = pd.DataFrame(combined_initial_info,columns = ["name", "image_path","dataset"])

In [26]:
df.head()

,name,image_path,dataset
0,14.jpeg,dataset\original_cityscapes_inputs\14.jpeg,Cityscapes
1,87.jpeg,dataset\original_cityscapes_inputs\87.jpeg,Cityscapes
2,189.jpeg,dataset\original_cityscapes_inputs\189.jpeg,Cityscapes
3,167.jpeg,dataset\original_cityscapes_inputs\167.jpeg,Cityscapes
4,439.jpeg,dataset\original_cityscapes_inputs\439.jpeg,Cityscapes


# Synthia color transformation code (for later use)

In [14]:
synthia_colors = [[  0,   0,   0], # void
         [70,130, 180], # sky
         [70,70,70], # building
        [128, 64, 128], # road
        [244, 35, 232], # sidewalk
         [64,64,128], # fense
         [107,142,35], # vegetation	
        [153, 153, 153], # pole
        [0, 0, 142], # car
        [220, 220, 0],  # traffic sign
        [220, 20, 60], # pedestrian
        [119, 11, 32], # bicycle
        [0, 0, 230], # motorcycle
        [250,170,160], # parking-slot
        [128,64,64], # road-work
        [250,170,30], # traffic light
        [152, 251, 152], # terrain
        [255, 0, 0], # rider
        [0, 0, 70], # truck
        [0, 60, 100], # bus
        [0, 80, 100], # train
        [102, 102, 156]# wall, lanemarking
    ]


category_map = {
    0: 0,
    1: 4,
    2: 0,
    3: 1,
    4: 2,
    5: 0,
    6: 3,
    7: 0,
    8: 5,
    9: 0,
    10: 0,
    11: 0,
    12: 0,
    13: 0,
    14: 0,
    15: 0,
    16: 0,
    17: 0,
    18: 0,
    19: 0,
    20: 0,
    21: 0}

In [35]:
# color to class
def color_to_class(label):
    # create new empty mask
    mask = np.zeros(shape=(label.shape[0], label.shape[1]), dtype = np.int32)
    # iterate through two dimensions
    for row in range(label.shape[0]):
        for col in range(label.shape[1]):
            a = label[row, col,:]
            # distance between this pixel and the original pixel
            d = cdist(np.array([a]),np.array(synthia_colors))
            idx = np.argmin(d)
            new_idx = category_map[idx]
            mask[row, col] = new_idx
    mask = np.reshape(mask, (mask.shape[0], mask.shape[1]))
    return mask

In [36]:
# class to color: useful for displaying the similar images later
colors = [[  0,   0,   0],
          [128, 64, 128],# road
          [244, 35, 232], # sidewalk
          [107, 142, 35],# vegetation
          [70, 130, 180], # sky
          [0, 0, 142], # car
         ]

def class_to_color(labels):
    label_colors = np.zeros((256,256,3))
    
    for i,row in enumerate(labels):
        for j,pixel in enumerate(row):
            label_colors[i,j] = colors[pixel]
    
    return label_colors.astype(int)  # make each pixel value an integer to visualize it better

# label_path

Cityscapes labels

In [29]:
# relative paths for cityscapes
cityscpaes_save_labels_path = "D:\zsh\graduation\Graduation-project-domain-shift-image-2-image\dataset\cityscapes_labels_sample"
cityscapes_label_folder_relative = os.path.relpath(cityscpaes_save_labels_path, start)

**For now: still use the sample of labels. Possibly changing to something else later on**

In [25]:
# labels of cityscapes
pickle_file = os.path.join("D:\zsh\graduation\Graduation-project-domain-shift-image-2-image\dataset\\5_classes_preprocessed", "validation_label_classes.pkl")
with open(pickle_file, 'rb') as f:
    Y_test = pickle.load(f)
    
cityscapes_labels = np.array(Y_test)

Generate and save labels for the subset for displaying in the VA system

In [38]:
cityscapes_labels_sample= []
cityscapes_label_path_sample = []

for name in cityscapes_names_sample:
    ind = int(name.split('.')[0])
    label = cityscapes_labels[ind]
    label_path = cityscapes_label_folder_relative+"\\"+name
    
    cityscapes_labels_sample.append(label)
    cityscapes_label_path_sample.append(label_path)

cityscapes_labels_sample = np.array(cityscapes_labels_sample)
cityscapes_label_path_sample = np.array(cityscapes_label_path_sample)

In [40]:
for i in range(sample_number):
    name = cityscapes_names_sample[i]
    label = cityscapes_labels_sample[i]
    label_color = class_to_color(label)
    label_image = Image.fromarray(label_color.astype(np.uint8))
    label_image.save(cityscpaes_save_labels_path+"\\"+name)

In [34]:
cityscapes_label_path_sample.shape

(100,)

Synthia labels

In [53]:
synthia_label_original_folder = "D:\zsh\graduation\Graduation-project-domain-shift-image-2-image\dataset\SYNTHIA_256\label-rgb"
synthia_save_labels_path = "D:\zsh\graduation\Graduation-project-domain-shift-image-2-image\dataset\SYNTHIA_256\\new_labels"
synthia_label_folder_relative = os.path.relpath(synthia_save_labels_path, start)

In [42]:
# def transform_masks_synthia(labels):
#     masks = []
#     for label in labels:
#         mask = color_to_class(label)
#         masks.append(mask)
#     masks = np.array(masks)

#     return masks

Note: comment out the corresponding lines if loading the synthia_labels_sample.pkl

In [60]:
# this usually takes a long time
synthia_labels_sample= []
synthia_label_path_sample = []

for name in synthia_names_sample:
    original_label_rgb = Image.open(synthia_label_original_folder+"\\"+name).convert("RGB")
    original_label_rgb = np.array(original_label_rgb)
    label = color_to_class(original_label_rgb)
    resized_label_rgb = class_to_color(label)
    label_path =synthia_label_folder_relative+"\\"+name
    resized_label_rgb = Image.fromarray(resized_label_rgb.astype(np.uint8))
    
#     save the new labels
    resized_label_rgb.save(synthia_save_labels_path+"\\"+name)
    
    synthia_labels_sample.append(label)
    synthia_label_path_sample.append(label_path)

synthia_labels_sample = np.array(synthia_labels_sample)
synthia_label_path_sample = np.array(synthia_label_path_sample)

In [62]:
len(synthia_label_path_sample)

100

save the synthia masks in a pickle to save effort

In [68]:
# with open('synthia_labels_sample.pkl', 'wb') as file:
      
#     # A new file will be created
#     pickle.dump(synthia_labels_sample, file)

with open('synthia_labels_sample.pkl', 'rb') as f:
    synthia_labels_sample = pickle.load(f)

Combine cityscapes_label_path_sample and synthia_label_path_sample

In [63]:
label_path_sample = np.concatenate((cityscapes_label_path_sample,synthia_label_path_sample),axis = 0)

In [65]:
df["label_path"] = label_path_sample

In [67]:
df.iloc[95:105]

,name,image_path,dataset,label_path
95,475.jpeg,dataset\original_cityscapes_inputs\475.jpeg,Cityscapes,dataset\cityscapes_labels_sample\475.jpeg
96,218.jpeg,dataset\original_cityscapes_inputs\218.jpeg,Cityscapes,dataset\cityscapes_labels_sample\218.jpeg
97,235.jpeg,dataset\original_cityscapes_inputs\235.jpeg,Cityscapes,dataset\cityscapes_labels_sample\235.jpeg
98,242.jpeg,dataset\original_cityscapes_inputs\242.jpeg,Cityscapes,dataset\cityscapes_labels_sample\242.jpeg
99,222.jpeg,dataset\original_cityscapes_inputs\222.jpeg,Cityscapes,dataset\cityscapes_labels_sample\222.jpeg
100,0001480.png,dataset\SYNTHIA_256\image\0001480.png,Synthia,dataset\SYNTHIA_256\new_labels\0001480.png
101,0003215.png,dataset\SYNTHIA_256\image\0003215.png,Synthia,dataset\SYNTHIA_256\new_labels\0003215.png
102,0002457.png,dataset\SYNTHIA_256\image\0002457.png,Synthia,dataset\SYNTHIA_256\new_labels\0002457.png
103,0004959.png,dataset\SYNTHIA_256\image\0004959.png,Synthia,dataset\SYNTHIA_256\new_labels\0004959.png
104,0001305.png,dataset\SYNTHIA_256\image\0001305.png,Synthia,dataset\SYNTHIA_256\new_labels\0001305.png


# similar_image_paths，similar_IoU_score 
(Note: called IoU_score originally): finding the most similar masks from another dataset

In [70]:
def most_similar_mask(instance, data,mode = "IoU"):
#     nodes = np.asarray(nodes)
    # Euclidean distance calculation
    if mode == "IoU":
        iou_list = []
        for new_instance in data:
#             the original IoU methods were not good because when both class is 0, the intersection won't count it
#             intersection = np.logical_and(instance, new_instance)
#             union = np.logical_or(instance, new_instance)
            intersection = len(np.where(instance == new_instance)[0])
            union = instance.shape[0]*instance.shape[1]
#             print(intersection)
#             print(union)
            iou_score = intersection / union
            iou_list.append(iou_score)
        iou_arr = np.array(iou_list)
        best_index = np.argmax(iou_arr)
        best_score = iou_arr[best_index]
    
    return best_index, best_score

In [75]:
cityscapes_similar_image_paths = []
cityscapes_similar_scores = []

for label in cityscapes_labels_sample:
    image_index_synthia,similar_score = most_similar_mask(label,synthia_labels_sample)
    similar_image_path = synthia_initial_info[image_index_synthia][1]
    
    cityscapes_similar_image_paths.append(similar_image_path)
    cityscapes_similar_scores.append(similar_score)

cityscapes_similar_image_paths = np.array(cityscapes_similar_image_paths)
cityscapes_similar_scores = np.array(cityscapes_similar_scores)

AttributeError: module 'numpy' has no attribute 'arra'

In [83]:
synthia_similar_image_paths = []
synthia_similar_scores = []

for label in synthia_labels_sample:
    image_index_cityscapes,similar_score = most_similar_mask(label,cityscapes_labels_sample)
    similar_image_path = cityscapes_initial_info[image_index_cityscapes][1]
    
    synthia_similar_image_paths.append(similar_image_path)
    synthia_similar_scores.append(similar_score)

synthia_similar_image_paths = np.array(synthia_similar_image_paths)
synthia_similar_scores = np.array(synthia_similar_scores)

In [87]:
similar_image_paths =  np.concatenate((cityscapes_similar_image_paths,synthia_similar_image_paths),axis = 0)
similar_scores = np.concatenate((cityscapes_similar_scores,synthia_similar_scores),axis = 0)

In [88]:
df["similar_image_paths"] = similar_image_paths
df["similar_IoU_score"]=similar_scores

In [91]:
df.iloc[95:105]

,name,image_path,dataset,label_path,similar_image_paths,similar_IoU_score
95,475.jpeg,dataset\original_cityscapes_inputs\475.jpeg,Cityscapes,dataset\cityscapes_labels_sample\475.jpeg,dataset\SYNTHIA_256\image\0004421.png,0.629333
96,218.jpeg,dataset\original_cityscapes_inputs\218.jpeg,Cityscapes,dataset\cityscapes_labels_sample\218.jpeg,dataset\SYNTHIA_256\image\0002688.png,0.541000
97,235.jpeg,dataset\original_cityscapes_inputs\235.jpeg,Cityscapes,dataset\cityscapes_labels_sample\235.jpeg,dataset\SYNTHIA_256\image\0005321.png,0.609619
98,242.jpeg,dataset\original_cityscapes_inputs\242.jpeg,Cityscapes,dataset\cityscapes_labels_sample\242.jpeg,dataset\SYNTHIA_256\image\0005321.png,0.502472
99,222.jpeg,dataset\original_cityscapes_inputs\222.jpeg,Cityscapes,dataset\cityscapes_labels_sample\222.jpeg,dataset\SYNTHIA_256\image\0004607.png,0.710312
100,0001480.png,dataset\SYNTHIA_256\image\0001480.png,Synthia,dataset\SYNTHIA_256\new_labels\0001480.png,dataset\original_cityscapes_inputs\237.jpeg,0.435898
101,0003215.png,dataset\SYNTHIA_256\image\0003215.png,Synthia,dataset\SYNTHIA_256\new_labels\0003215.png,dataset\original_cityscapes_inputs\188.jpeg,0.492340
102,0002457.png,dataset\SYNTHIA_256\image\0002457.png,Synthia,dataset\SYNTHIA_256\new_labels\0002457.png,dataset\original_cityscapes_inputs\147.jpeg,0.487686
103,0004959.png,dataset\SYNTHIA_256\image\0004959.png,Synthia,dataset\SYNTHIA_256\new_labels\0004959.png,dataset\original_cityscapes_inputs\101.jpeg,0.587204
104,0001305.png,dataset\SYNTHIA_256\image\0001305.png,Synthia,dataset\SYNTHIA_256\new_labels\0001305.png,dataset\original_cityscapes_inputs\423.jpeg,0.536331


In [92]:
# df.to_csv("system_df_full.csv",index=True)

# generate from list of images and labels

* class distribution
    * (other_ratio, road_ratio, sidewalk_ratio, vegetation_ratio, sky_ratio, car_ratio)
* embedding of input (originally: tsne_1, tsne_2)
    * simple dimensionality reduction (simple_tsne_1,simple_tsne_2)
    * and also the embedding from classification model(meaningful_tsne_1,meaningful_tsne_1)

Load images to data loader

In [94]:
class CityscapesDataset(Dataset):
    
    def __init__(self, images, labels, noise_level = 0):
        self.images = images
        self.labels = labels
        self.noise_level = noise_level
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
        if self.noise_level!=0:
            image = image+(self.noise_level*np.random.normal(0, (image.max() - image.min()), image.shape)).astype("uint8") # (mean, sigma, image_shape)
        image = self.transform(image)
        label = torch.from_numpy(label).long()
        return image, label
        
    def transform(self, image):
        transform_ops = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)) # normalize to control the "dynamic range" of activations of different layers
        ])
        return transform_ops(image)

In [96]:
repo_path = "D:\zsh\graduation\\Graduation-project-domain-shift-image-2-image"
# cityscapes_label_path = "dataset\\cityscapes_labels_100"
cityscapes_images = []
cityscapes_labels = []
synthia_images = []
synthia_labels = []

for index,path in enumerate(df["image_path"]):
    full_path = os.path.join(repo_path,path)
    dataset = df["dataset"].iloc[index]
    # parse to get the name of image
    name = os.path.split(path)[1]
    label_index = int(name.split(".")[0])
    if dataset == "Cityscapes":
        # image
        img = Image.open(full_path)
        img = np.array(img)
        # label
        label = Y_test[label_index]
        label = np.array(label)
        cityscapes_images.append(img)
        cityscapes_labels.append(label)
    elif dataset == "Synthia":
        # image
        img = Image.open(full_path)
        img = np.array(img)
        # label
        label = synthia_labels_sample[index-100]
        label = np.array(label)
        synthia_images.append(img)
        synthia_labels.append(label)
    

In [116]:
cityscapes_images = np.array(cityscapes_images)
cityscapes_labels = np.array(cityscapes_labels)
synthia_images = np.array(synthia_images)
synthia_labels = np.array(synthia_labels)

print(synthia_images.shape)
print(synthia_images.shape)

# load to Dataset class and DataLoader
batch_size = 1
cityscapes_dataset = CityscapesDataset(cityscapes_images, cityscapes_labels,noise_level=0)
cityscapes_loader = DataLoader(cityscapes_dataset, batch_size=batch_size)

synthia_dataset = CityscapesDataset(synthia_images, synthia_labels,noise_level=0)
synthia_loader = DataLoader(synthia_dataset, batch_size=batch_size)

(100, 256, 256, 3)
(100, 256, 256, 3)


**class distribution**

In [109]:
def get_class_distribution(labels):
    class_dist = []
    for label in labels:
        class_for_label = []
        for i in range(6):
            element_count = np.count_nonzero(label==i)
            class_for_label.append(element_count/(256*256))
        class_dist.append(class_for_label)
        class_for_label= class_for_label# change to another size if the image size is changed
    class_dist = np.array(class_dist)
    return class_dist

In [114]:
cityscpaes_class_dist = get_class_distribution(cityscapes_labels)
# synthia_labels
synthia_class_dist = get_class_distribution(synthia_labels)

In [115]:
print(cityscpaes_class_dist.shape)
print(synthia_class_dist.shape)

(100, 6)
(100, 6)


In [118]:
class_dist = np.concatenate((cityscpaes_class_dist,synthia_class_dist),axis=0)

In [119]:
class_dist_df = pd.DataFrame(class_dist,columns=["other_ratio","road_ratio","sidewalk_ratio","vegetation_ratio","sky_ratio","car_ratio"])
class_dist_df.head()

,other_ratio,road_ratio,sidewalk_ratio,vegetation_ratio,sky_ratio,car_ratio
0,0.268173,0.380219,0.054855,0.259460,0.026566,0.010727
1,0.390427,0.322311,0.000000,0.128143,0.004623,0.154495
2,0.615875,0.264603,0.010590,0.037872,0.023697,0.047363
3,0.235931,0.296295,0.021698,0.421646,0.023544,0.000885
4,0.501968,0.312622,0.026047,0.157547,0.000031,0.001785


In [125]:
df_full = pd.concat((df,class_dist_df),axis=1)

In [127]:
df_full.to_csv("system_df_full.csv",index=True)

**embedding of input**

* simple dimensionality reduction (simple_tsne_1,simple_tsne_2)
* and also the embedding from classification model(meaningful_tsne_1,meaningful_tsne_1)

In [128]:
# simple dimensionality reduction
combined_images = np.concatenate((cityscapes_images,synthia_images),axis=0)

In [132]:
simple_combined_embedding =np.reshape(combined_images,(len(combined_images), 256*256*3))

pca_50 = PCA(n_components=50)
pca_embedding = pca_50.fit_transform(simple_combined_embedding)
print(np.sum(pca_50.explained_variance_ratio_))
tsne = TSNE()
simple_tsne_embedding = tsne.fit_transform(pca_embedding)

0.7971421033590691


In [136]:
cd D:\zsh\graduation\ViTs-vs-CNNs

D:\zsh\graduation\ViTs-vs-CNNs


In [137]:
import torch
import torch.nn as nn
import global_val

from models.ghost_bn import GhostBN2D_ADV
from models.advresnet_gbn_gelu import Affine
import models.advresnet_gbn_gelu as advres
from main_adv_res import EightBN

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

classifier_model = advres.__dict__["resnet50"](norm_layer = EightBN)
weights_path = "D:\zsh\graduation\Graduation-project-domain-shift-image-2-image\models\\advres50_gelu.pth"
weight_dict = torch.load(weights_path,map_location=device)["model"]

classifier_model.load_state_dict(weight_dict)

<All keys matched successfully>

In [138]:
cd D:\zsh\graduation\Graduation-project-domain-shift-image-2-image\jupyter

D:\zsh\graduation\Graduation-project-domain-shift-image-2-image


In [145]:
def get_representation_for_image(image,label):
    # resize the image and transform it to tensor # [256,256,3] -> [224,224,3] 
    resized_image = resize(image, (224, 224,3))
    resized_image_tensor = torch.from_numpy(resized_image)
    resized_image_tensor = torch.permute(resized_image_tensor, (2, 0, 1))
    resized_image_tensor = resized_image_tensor[None,:] # [3,224,224] -> [1,3,224,224] (because model takes 4D input)
    
    # dictionary for the activations
    activations = {}

    def get_activations(name):
        def hook(model, input, output):
            activations[name] = output.detach()
        return hook

    # use the output of (avgpool) layer because it is the same as the input of last layer
    h = classifier_model.avgpool.register_forward_hook(get_activations("input_last_layer"))

    classifier_model.eval()
    classifier_model.sing=True
    classifier_model.training=False
    out = classifier_model(resized_image_tensor.float(),label)
    
    # remove the hook
    h.remove()

    first_part_embedding = torch.flatten(activations["input_last_layer"][0])
    # the second part of the embedding is size of each class
    #  (this section of code is overlapping with generating class distribution, improve this for a better result)   
#     second_part_embedding=[]
#     for i in range(6):
#         class_size = np.count_nonzero(label == i)
#         second_part_embedding.append(class_size)
#     embedding = np.concatenate((first_part_embedding.numpy(),np.array(second_part_embedding)),axis = 0)
    
    return first_part_embedding

In [146]:
embedding_list = []
for i, image in enumerate(combined_images): # i is not used, could remove the enumerate
    first_part_embedding = get_representation_for_image(image,"__") # y_label is never used, so just use replacement
    second_part_embedding = class_dist[i]
    embedding = np.concatenate((first_part_embedding.numpy(),np.array(second_part_embedding)),axis = 0)
    embedding_list.append(embedding)

In [149]:
embedding_arr = np.array(embedding_list)
pca_50 = PCA(n_components=50)
pca_embedding = pca_50.fit_transform(embedding_arr)
print(np.sum(pca_50.explained_variance_ratio_))
# pca = PCA(n_components=2)
tsne = TSNE()
meaningful_tsne_embedding = tsne.fit_transform(pca_embedding)

0.8664894254259768


In [152]:
full_tsne_embedding = np.concatenate((simple_tsne_embedding,meaningful_tsne_embedding),axis=1)

In [154]:
df_input_embedding = pd.DataFrame(full_tsne_embedding, columns=["simple_tsne_1","simple_tsne_2","meaningful_tsne_1","meaningful_tsne_2"])

In [156]:
df_full = pd.concat((df_full,df_input_embedding),axis = 1)
df_full.head()

,name,image_path,dataset,label_path,similar_image_paths,similar_IoU_score,other_ratio,road_ratio,sidewalk_ratio,vegetation_ratio,sky_ratio,car_ratio,simple_tsne_1,simple_tsne_2,meaningful_tsne_1,meaningful_tsne_2
0,14.jpeg,dataset\original_cityscapes_inputs\14.jpeg,Cityscapes,dataset\cityscapes_labels_sample\14.jpeg,dataset\SYNTHIA_256\image\0002688.png,0.475754,0.268173,0.380219,0.054855,0.259460,0.026566,0.010727,2.444247,-4.093524,6.933020,7.210295
1,87.jpeg,dataset\original_cityscapes_inputs\87.jpeg,Cityscapes,dataset\cityscapes_labels_sample\87.jpeg,dataset\SYNTHIA_256\image\0002688.png,0.480301,0.390427,0.322311,0.000000,0.128143,0.004623,0.154495,-1.070811,4.178267,4.218904,12.409754
2,189.jpeg,dataset\original_cityscapes_inputs\189.jpeg,Cityscapes,dataset\cityscapes_labels_sample\189.jpeg,dataset\SYNTHIA_256\image\0004607.png,0.653656,0.615875,0.264603,0.010590,0.037872,0.023697,0.047363,-3.867693,1.602131,-1.544376,10.230064
3,167.jpeg,dataset\original_cityscapes_inputs\167.jpeg,Cityscapes,dataset\cityscapes_labels_sample\167.jpeg,dataset\SYNTHIA_256\image\0005291.png,0.453262,0.235931,0.296295,0.021698,0.421646,0.023544,0.000885,0.109568,-2.603335,3.992030,5.020967
4,439.jpeg,dataset\original_cityscapes_inputs\439.jpeg,Cityscapes,dataset\cityscapes_labels_sample\439.jpeg,dataset\SYNTHIA_256\image\0002688.png,0.522873,0.501968,0.312622,0.026047,0.157547,0.000031,0.001785,3.045528,-4.438342,8.317305,5.757763


In [157]:
df_full.to_csv("system_df_full.csv",index=True)

#  bottleneck_activations_embedding, prediction_path, and performance
* save output locally, and then add path (prediction_path)
* performance: (other_IoU, road_IoU, sidewalk_IoU, vegetation_IoU, sky_IoU, car_IoU)
    **How do you get the IoU per class (check original paper)**

Load model

In [158]:
class UNet(nn.Module):
    
    def __init__(self, num_classes):
        super(UNet, self).__init__()
        self.num_classes = num_classes
        self.contracting_11 = self.conv_block(in_channels=3, out_channels=64)
        self.contracting_12 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.contracting_21 = self.conv_block(in_channels=64, out_channels=128)
        self.contracting_22 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.contracting_31 = self.conv_block(in_channels=128, out_channels=256)
        self.contracting_32 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.contracting_41 = self.conv_block(in_channels=256, out_channels=512)
        self.contracting_42 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.middle = self.conv_block(in_channels=512, out_channels=1024)
        self.expansive_11 = nn.ConvTranspose2d(in_channels=1024, out_channels=512, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.expansive_12 = self.conv_block(in_channels=1024, out_channels=512)
        self.expansive_21 = nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.expansive_22 = self.conv_block(in_channels=512, out_channels=256)
        self.expansive_31 = nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.expansive_32 = self.conv_block(in_channels=256, out_channels=128)
        self.expansive_41 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.expansive_42 = self.conv_block(in_channels=128, out_channels=64)
        self.output = nn.Conv2d(in_channels=64, out_channels=num_classes, kernel_size=3, stride=1, padding=1)
        
    def conv_block(self, in_channels, out_channels):
        block = nn.Sequential(nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(num_features=out_channels),
                                    nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(num_features=out_channels))
        return block
    
    def forward(self, X):
        contracting_11_out = self.contracting_11(X) # [-1, 64, 256, 256]
        contracting_12_out = self.contracting_12(contracting_11_out) # [-1, 64, 128, 128]
        contracting_21_out = self.contracting_21(contracting_12_out) # [-1, 128, 128, 128]
        contracting_22_out = self.contracting_22(contracting_21_out) # [-1, 128, 64, 64]
        contracting_31_out = self.contracting_31(contracting_22_out) # [-1, 256, 64, 64]
        contracting_32_out = self.contracting_32(contracting_31_out) # [-1, 256, 32, 32]
        contracting_41_out = self.contracting_41(contracting_32_out) # [-1, 512, 32, 32]
        contracting_42_out = self.contracting_42(contracting_41_out) # [-1, 512, 16, 16]
        middle_out = self.middle(contracting_42_out) # [-1, 1024, 16, 16]
        expansive_11_out = self.expansive_11(middle_out) # [-1, 512, 32, 32]
        expansive_12_out = self.expansive_12(torch.cat((expansive_11_out, contracting_41_out), dim=1)) # [-1, 1024, 32, 32] -> [-1, 512, 32, 32]
        expansive_21_out = self.expansive_21(expansive_12_out) # [-1, 256, 64, 64]
        expansive_22_out = self.expansive_22(torch.cat((expansive_21_out, contracting_31_out), dim=1)) # [-1, 512, 64, 64] -> [-1, 256, 64, 64]
        expansive_31_out = self.expansive_31(expansive_22_out) # [-1, 128, 128, 128]
        expansive_32_out = self.expansive_32(torch.cat((expansive_31_out, contracting_21_out), dim=1)) # [-1, 256, 128, 128] -> [-1, 128, 128, 128]
        expansive_41_out = self.expansive_41(expansive_32_out) # [-1, 64, 256, 256]
        expansive_42_out = self.expansive_42(torch.cat((expansive_41_out, contracting_11_out), dim=1)) # [-1, 128, 256, 256] -> [-1, 64, 256, 256]
        output_out = self.output(expansive_42_out) # [-1, num_classes, 256, 256]
        return output_out

In [159]:
model_path = "D:\zsh\graduation\Graduation-project-domain-shift-image-2-image\models\\5-classes-U-Net-2023-03-09.pth"

device = torch.device('cpu')
model_ = UNet(num_classes=6).to(device)
model_.load_state_dict(torch.load(model_path,map_location=device))

<All keys matched successfully>

In [186]:
def IoU_per_class(prediction,label):
    prediction = prediction.detach().numpy()
    label = label.detach().numpy()
    # Loop over each class
    iou_list = []
    # Flatten label and class arrays
    flat_prediction = prediction.flatten()
    flat_label = label.flatten()

    for i in range(6):
        
        # Calculate intersection and union
        intersection = np.sum((flat_prediction == i) & (flat_label == i))
        union = np.sum((flat_prediction == i) | (flat_label == i))

        # Calculate IoU
        iou = intersection / (union + 1e-12)

        # Store IoU value in dictionary
        iou_list.append(iou)
    
    # calculate overall IoU
    intersection = len(np.where(flat_prediction == flat_label)[0])
    union = len(flat_label)
    overall_iou = intersection / (union + 1e-12)
    
    return overall_iou,iou_list

In [250]:
def getActivation(name):
    # the hook signature
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

cityscapes_activations = []
cityscapes_overall_iou = []
cityscapes_iou_by_class =[]
cityscapes_predictions = []

for image,label in cityscapes_loader:
    activation = {}
    h = model_.middle[3].register_forward_hook(getActivation("bottleneck"))
    out = model_(image)
    prediction = torch.argmax(out, dim=1)
    h.remove()
    overall_iou,iou_list = IoU_per_class(prediction,label)
    cityscapes_overall_iou.append(overall_iou)
    cityscapes_iou_by_class.append(iou_list)
    prediction = prediction.detach().numpy()[0]
    cityscapes_predictions.append(prediction)
    instance_activation = activation["bottleneck"]
#     print(instance_activation.size())
    # need to reshape
    instance_activation_reshaped = instance_activation.clone().squeeze(0).reshape(256,1024).numpy()
    cityscapes_activations.append(instance_activation_reshaped)
#     print(instance_activation_reshaped.shape)

In [251]:
cityscapes_activations = np.array(cityscapes_activations)
cityscapes_overall_iou = np.array(cityscapes_overall_iou)
cityscapes_iou_by_class = np.array(cityscapes_iou_by_class)
cityscapes_predictions = np.array(cityscapes_predictions)

In [252]:
cityscapes_predictions.shape

(100, 256, 256)

In [ ]:
cityscapes_predictions.size()

In [254]:
synthia_activations = []
synthia_overall_iou = []
synthia_iou_by_class =[]
synthia_predictions =[]
for image,label in synthia_loader:
    activation = {}
    h = model_.middle[3].register_forward_hook(getActivation("bottleneck"))
    out = model_(image)
    prediction = torch.argmax(out, dim=1)
    h.remove()
    overall_iou,iou_list = IoU_per_class(prediction,label)
    synthia_overall_iou.append(overall_iou)
    synthia_iou_by_class.append(iou_list)
    prediction = prediction.detach().numpy()[0]
    synthia_predictions.append(prediction)
    instance_activation = activation["bottleneck"]
#     print(instance_activation.size())
    # need to reshape
    instance_activation_reshaped = instance_activation.clone().squeeze(0).reshape(256,1024).numpy()
    synthia_activations.append(instance_activation_reshaped)

synthia_activations = np.array(synthia_activations)
synthia_overall_iou = np.array(synthia_overall_iou)
synthia_iou_by_class = np.array(synthia_iou_by_class)

In [255]:
combined_activations = np.concatenate((cityscapes_activations,synthia_activations),axis=0)
combined_overall_iou = np.concatenate((cityscapes_overall_iou,synthia_overall_iou),axis=0)
combined_iou_by_class = np.concatenate((cityscapes_iou_by_class,synthia_iou_by_class),axis=0)
combined_predictions = np.concatenate((cityscapes_predictions,synthia_predictions),axis=0)

In [259]:
combined_predictions.shape

(200, 256, 256)

Save predictions to paths 

In [265]:
cityscapes_prediction_folder = "D:\zsh\graduation\Graduation-project-domain-shift-image-2-image\dataset\cityscapes_predictions"
cityscapes_prediction_folder_relative = os.path.relpath(cityscapes_prediction_folder, start)

synthia_prediction_folder = "D:\zsh\graduation\Graduation-project-domain-shift-image-2-image\dataset\SYNTHIA_256\predictions"
synthia_prediction_folder_relative = os.path.relpath(synthia_prediction_folder, start)

In [271]:
prediction_paths = []
for i, prediction in enumerate(combined_predictions):
    name = df.iloc[i]["name"]
    dataset = df.iloc[i]["dataset"]
    prediction_img = class_to_color(prediction)
    if dataset=="Cityscapes":
        full_folder = cityscapes_prediction_folder
        relative_folder = cityscapes_prediction_folder_relative
    elif dataset=="Synthia":
        full_folder = synthia_prediction_folder
        relative_folder = synthia_prediction_folder_relative
    
    prediction_img = Image.fromarray(prediction_img.astype(np.uint8))
    prediction_img.save(full_folder+"\\"+name)
    prediction_paths.append(relative_folder+"\\"+name)

* activations

(next cell has 6 minute run time for 200 images)

In [216]:
# dimensionality reduction for combined instance activations
combined_activations_2d = np.reshape(combined_activations,(200*256,1024))

pca = PCA(n_components=50)
pca_embedding = pca.fit_transform(combined_activations_2d)
activations_tsne_embedding = TSNE(n_components=2).fit_transform(pca_embedding)

In [218]:
activations_tsne_embedding_reshaped = np.reshape(activations_tsne_embedding,(200,256,2))

In [237]:
with open('activations_embeddings.pkl', 'wb') as file:
      
    # A new file will be created
    pickle.dump(activations_tsne_embedding_reshaped, file)

In [217]:
activations_tsne_embedding.shape

(51200, 2)

In [221]:
# activation_df
df_full["bottleneck_activations_embedding"] = activations_tsne_embedding_reshaped.tolist()

# overall_iou
df_full["overall_iou"] = combined_overall_iou

# iou by class
iou_class_df = pd.DataFrame(combined_iou_by_class,columns = ["other_iou","road_iou","sidewalk_iou","vegetation_iou","sky_iou","car_iou"])
df_full = pd.concat((df_full,iou_class_df),axis=1)

In [275]:
df_full["prediction_path"] = prediction_paths

In [280]:
df_rearrange = df_full[['name','dataset', 'image_path', 'label_path','prediction_path', 'similar_image_paths','similar_IoU_score', 
                       'other_ratio', 'road_ratio', 'sidewalk_ratio','vegetation_ratio', 'sky_ratio', 'car_ratio', 
                       'simple_tsne_1','simple_tsne_2', 'meaningful_tsne_1', 'meaningful_tsne_2',
       'bottleneck_activations_embedding', 'overall_iou', 'other_iou',
       'road_iou', 'sidewalk_iou', 'vegetation_iou', 'sky_iou', 'car_iou']]

In [277]:
df_full.columns

Index(['name', 'image_path', 'dataset', 'label_path', 'similar_image_paths',
       'similar_IoU_score', 'other_ratio', 'road_ratio', 'sidewalk_ratio',
       'vegetation_ratio', 'sky_ratio', 'car_ratio', 'simple_tsne_1',
       'simple_tsne_2', 'meaningful_tsne_1', 'meaningful_tsne_2',
       'bottleneck_activations_embedding', 'overall_iou', 'other_iou',
       'road_iou', 'sidewalk_iou', 'vegetation_iou', 'sky_iou', 'car_iou',
       'prediction_path'],
      dtype='object')

In [281]:
df_rearrange

,name,dataset,image_path,label_path,prediction_path,similar_image_paths,similar_IoU_score,other_ratio,road_ratio,sidewalk_ratio,...,meaningful_tsne_1,meaningful_tsne_2,bottleneck_activations_embedding,overall_iou,other_iou,road_iou,sidewalk_iou,vegetation_iou,sky_iou,car_iou
0,14.jpeg,Cityscapes,dataset\original_cityscapes_inputs\14.jpeg,dataset\cityscapes_labels_sample\14.jpeg,dataset\cityscapes_predictions\14.jpeg,dataset\SYNTHIA_256\image\0002688.png,0.475754,0.268173,0.380219,0.054855,...,6.933020,7.210295,"[[9.249624252319336, 25.688404083251953], [-24...",0.880295,0.719708,0.944721,0.695616,0.718850,0.862466,0.311558
1,87.jpeg,Cityscapes,dataset\original_cityscapes_inputs\87.jpeg,dataset\cityscapes_labels_sample\87.jpeg,dataset\cityscapes_predictions\87.jpeg,dataset\SYNTHIA_256\image\0002688.png,0.480301,0.390427,0.322311,0.000000,...,4.218904,12.409754,"[[9.378799438476562, 25.773765563964844], [-23...",0.905487,0.788071,0.962711,0.000000,0.739955,0.164607,0.863164
2,189.jpeg,Cityscapes,dataset\original_cityscapes_inputs\189.jpeg,dataset\cityscapes_labels_sample\189.jpeg,dataset\cityscapes_predictions\189.jpeg,dataset\SYNTHIA_256\image\0004607.png,0.653656,0.615875,0.264603,0.010590,...,-1.544376,10.230064,"[[9.018597602844238, 25.574705123901367], [-24...",0.849472,0.780169,0.925425,0.127430,0.278427,0.892169,0.811478
3,167.jpeg,Cityscapes,dataset\original_cityscapes_inputs\167.jpeg,dataset\cityscapes_labels_sample\167.jpeg,dataset\cityscapes_predictions\167.jpeg,dataset\SYNTHIA_256\image\0005291.png,0.453262,0.235931,0.296295,0.021698,...,3.992030,5.020967,"[[10.460427284240723, 27.210540771484375], [-2...",0.691772,0.462270,0.940231,0.192641,0.402005,0.920625,0.008711
4,439.jpeg,Cityscapes,dataset\original_cityscapes_inputs\439.jpeg,dataset\cityscapes_labels_sample\439.jpeg,dataset\cityscapes_predictions\439.jpeg,dataset\SYNTHIA_256\image\0002688.png,0.522873,0.501968,0.312622,0.026047,...,8.317305,5.757763,"[[10.263081550598145, 25.39412498474121], [-23...",0.738602,0.669778,0.695943,0.133251,0.536060,0.000000,0.033378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0009204.png,Synthia,dataset\SYNTHIA_256\image\0009204.png,dataset\SYNTHIA_256\new_labels\0009204.png,dataset\SYNTHIA_256\predictions\0009204.png,dataset\original_cityscapes_inputs\188.jpeg,0.492752,0.414795,0.169708,0.330994,...,-0.063736,-16.626930,"[[12.014287948608398, 21.802379608154297], [-2...",0.303146,0.290828,0.224877,0.024413,0.088516,0.000000,0.000000
196,0000063.png,Synthia,dataset\SYNTHIA_256\image\0000063.png,dataset\SYNTHIA_256\new_labels\0000063.png,dataset\SYNTHIA_256\predictions\0000063.png,dataset\original_cityscapes_inputs\174.jpeg,0.481476,0.356415,0.243332,0.053818,...,-9.205073,-8.018494,"[[12.287400245666504, 25.470184326171875], [-2...",0.563889,0.383060,0.525158,0.087156,0.476979,0.426459,0.000000
197,0000582.png,Synthia,dataset\SYNTHIA_256\image\0000582.png,dataset\SYNTHIA_256\new_labels\0000582.png,dataset\SYNTHIA_256\predictions\0000582.png,dataset\original_cityscapes_inputs\248.jpeg,0.512939,0.256805,0.283630,0.038498,...,-10.830279,-8.390073,"[[13.566570281982422, 24.40737533569336], [-22...",0.565704,0.393866,0.488740,0.142928,0.387739,0.503347,0.000000
198,0005816.png,Synthia,dataset\SYNTHIA_256\image\0005816.png,dataset\SYNTHIA_256\new_labels\0005816.png,dataset\SYNTHIA_256\predictions\0005816.png,dataset\original_cityscapes_inputs\147.jpeg,0.430588,0.386841,0.029251,0.383972,...,0.840205,-6.707541,"[[10.129311561584473, 24.113561630249023], [-2...",0.364288,0.360000,0.016676,0.106556,0.315102,0.467021,0.000000


In [283]:
df_rearrange.to_csv("system_df_full.csv",index=True)